### Topic Modeling for Twitter

In this notebook I will apply topic modeling on the twitter data I cleaned in a separate notebook.
I will use LDA algorithm to model topic, 
Here are some usefuls ressources I used to learn the intuition behind topic modeling :
    [AV introduction to topic Modeling](https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/) 
    [LDA Video](https://www.youtube.com/watch?v=3mHy4OSyRf0) 
    [Topic Modeling With Gesim](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/)
For the first start I will use pyhon sklearn library and gensim library

In [91]:
# uncoment the next line to install gensim
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 359kB/s eta 0:00:01
     |████████████████████████████████| 184kB 5.4MB/s eta 0:00:01
     |████████████████████████████████| 235kB 15.4MB/s eta 0:00:01
     |████████████████████████████████| 829kB 10.4MB/s eta 0:00:01    |██▊                             | 71kB 10.6MB/s eta 0:00:01
     |████████████████████████████████| 61kB 7.0MB/s eta 0:00:011
     |████████████████████████████████| 92kB 6.2MB/s eta 0:00:011
  Stored in directory: /Users/espoir-mur/Library/Caches/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Stored in directory: /Users/espoir-mur/Library/Caches/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built pyLDAvis future


In [92]:
import pandas as pd
import gensim as gs
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from pprint import pprint
import pyLDAvis

In [45]:
cleanned_tweets = pd.read_csv('../data/cleanned_tweets.csv', index_col='Unnamed: 0')

In [46]:
cleanned_tweets.dropna(inplace=True)

In [47]:
tweets_array = cleanned_tweets.get('cleanned_tweet').values

In [48]:
tweets_array = [tweet.split(' ') for tweet in tweets_array]

In [49]:
# Create Dictionary

id2word = corpora.Dictionary(tweets_array)

In [50]:
# Create Corpus
texts = tweets_array

In [51]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]]


In [99]:
lda_model = gs.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=100,
                                           alpha='auto',
                                           per_word_topics=True)

In [101]:
for label, words in lda_model.print_topics():
    print('==================')
    print("topic label ", label, ' =>word ',  words)

topic label  0  =>word  0.017*"come" + 0.017*"mort" + 0.013*"@fcc_rdc" + 0.013*"@sindika_dokolo" + 0.011*"tshimboj" + 0.011*"devoir" + 0.011*"discussion" + 0.011*"outbreak" + 0.010*"@elogemwandwe" + 0.010*"news"
topic label  1  =>word  0.013*"@actualitecd" + 0.010*"around" + 0.010*"@martinfayulu" + 0.010*"décider" + 0.009*"population" + 0.009*"drcongo" + 0.009*"petit" + 0.009*"enfant" + 0.009*"kalemie" + 0.008*"cependant"
topic label  2  =>word  0.040*"@presidence_rdc" + 0.035*"tshisekedi" + 0.026*"de" + 0.021*"etienne" + 0.019*"national" + 0.017*"@topcongo" + 0.017*"@fatshi13" + 0.016*"jeudi" + 0.016*"mai" + 0.015*"rdcongo"
topic label  3  =>word  0.040*"le" + 0.033*"de" + 0.030*"-" + 0.020*"président" + 0.019*"non" + 0.018*"tshisekedi" + 0.015*"mai" + 0.015*"kabila" + 0.014*"étienne" + 0.013*"dépouill"
topic label  4  =>word  0.030*"être" + 0.028*"@moise_katumbi" + 0.022*"@fatshi13" + 0.021*"@vitalkamerhe1" + 0.021*"ce" + 0.017*"@abelamundala" + 0.013*"pays" + 0.013*"@mwema_y" + 0.01

From the first impression the model is not that accurate , there are some with word which has no sense, the only topic I can identify from those tweet are the holy day on thurday due to Tshisekedi death .

From now , if we remove mentions and english word and article we can have better accuracy, we need to improve the model
on that side

The step to improve the model will be to remove mention, remove english word, and some useless word.

In [102]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Perplexity:  -7.954524536656986


In [103]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4338861927091099


In [104]:
import pyLDAvis.gensim

In [105]:

vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/espoir-mur/Projects/Personnal/twitter-analysis/.venv3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.297407 -0.048672       1        1  19.828217
9      0.008862  0.219855       2        1  14.180862
7     -0.082190 -0.280250       3        1  13.563924
3      0.185385 -0.012118       4        1  11.749595
1     -0.107839  0.037006       5        1   9.287237
4     -0.068268  0.020338       6        1   7.684615
8     -0.070763 -0.004555       7        1   6.946169
6     -0.007222  0.030084       8        1   6.192646
0     -0.079707  0.021609       9        1   6.139990
5     -0.075665  0.016704      10        1   4.426743, topic_info=     Category        Freq             Term       Total  loglift  logprob
88    Default  178.000000  @presidence_rdc  178.000000  30.0000  30.0000
5     Default  182.000000               de  182.000000  29.0000  29.0000
49    Default  245.000000       tshisekedi  245.000000  28.0000  28.0000
15    Default  132.000000                -  132.000000  27.0000  27.0000
8     Default  187.000000               le  187.000000  26.0000  26.0000
146   Default   88.000000                e   88.000000  25.0000  25.0000
86    Default  101.000000        @fatshi13  101.000000  24.0000  24.0000
323   Default   65.000000            ebola   65.000000  23.0000  23.0000
59    Default   46.000000             être   46.000000  22.0000  22.0000
70    Default   88.000000          etienne   88.000000  21.0000  21.0000
196   Default  101.000000              mai  101.000000  20.0000  20.0000
264   Default   44.000000   @moise_katumbi   44.000000  19.0000  19.0000
234   Default   82.000000         national   82.000000  18.0000  18.0000
200   Default   57.000000          obsèque   57.000000  17.0000  17.0000
89    Default   70.000000        @topcongo   70.000000  16.0000  16.0000
207   Default   53.000000              dei   53.000000  15.0000  15.0000
209   Default   50.000000               di   50.000000  14.0000  14.0000
57    Default   54.000000        président   54.000000  13.0000  13.0000
205   Default   49.000000            conti   49.000000  12.0000  12.0000
614   Default   44.000000              non   44.000000  11.0000  11.0000
477   Default   61.000000          rdcongo   61.000000  10.0000  10.0000
104   Default   60.000000        kinshaser   60.000000   9.0000   9.0000
81    Default   33.000000               il   33.000000   8.0000   8.0000
102   Default   78.000000            jeudi   78.000000   7.0000   7.0000
195   Default   57.000000          journée   57.000000   6.0000   6.0000
757   Default   33.000000   @vitalkamerhe1   33.000000   5.0000   5.0000
686   Default   32.000000               se   32.000000   4.0000   4.0000
52    Default   33.000000               ce   33.000000   3.0000   3.0000
149   Default   42.000000         naufrage   42.000000   2.0000   2.0000
247   Default   46.000000          étienne   46.000000   1.0000   1.0000
...       ...         ...              ...         ...      ...      ...
93    Topic10   15.352538             héro   16.039797   3.0737  -4.0625
1203  Topic10   14.718591      ministériel   15.405807   3.0719  -4.1047
1204  Topic10   14.084079            poste   14.771273   3.0699  -4.1488
338   Topic10   10.459727      communiquer   11.147001   3.0539  -4.4463
455   Topic10   10.026967         @drc_2nd   10.714134   3.0512  -4.4885
1226  Topic10    9.852067          premier   10.539512   3.0501  -4.5061
2718  Topic10    9.043314          travers    9.730525   3.0443  -4.5918
2118  Topic10    8.514159         réserver    9.201357   3.0399  -4.6521
1483  Topic10    7.961676          engager    8.648870   3.0347  -4.7192
87    Topic10    7.365250     @olongobasil    8.052475   3.0283  -4.7970
97    Topic10    7.359841            élevé    8.047030   3.0282  -4.7978
1606  Topic10    6.888562            front    7.575837   3.0224  -4.8639
5525  Topic10    6.620796           souten    7.308022   3.0187  -4.9036
5524  Topic10    6.62079

### II. trying Mallet Model

Let first download the model!!

In [117]:
#Uncomment to download the model
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip -O ../.venv3/models/mallet.zip

--2019-05-30 20:17:45--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘../.venv3/models/mallet.zip’

../.venv3/models/ma 100%[===================>]  15.43M  1.58MB/s    in 12s     

2019-05-30 20:17:58 (1.24 MB/s) - ‘../.venv3/models/mallet.zip’ saved [16184794/16184794]



In [120]:
mallet_path = '../.venv3/models/mallet.zip' # update this path
#ldamallet = gs.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)